In [134]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import csv
import tensorflow as tf
import pandas as pd

In [135]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [136]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [137]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             #mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             #mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             #) 
    # Draw pose connections
#     mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
#                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
#                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
#                              ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [138]:
def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*2)
    rh = np.array([[res.x, res.y] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*2)
    return np.concatenate([lh, rh])

In [139]:
def select_mode(key, mode):
    number = -1
    if 48 <= key <= 57:  # 0 ~ 9
        number = key - 48
    if key == 110:  # n
        mode = 0
    if key == 107:  # k
        mode = 1
    
    return number,mode


In [140]:
def logging_csv(number, mode, landmark_list):
    if mode == 0:
        pass
    if mode == 1 and (0 <= number <= 9):
        csv_path = '/Users/richamodi/Desktop/ISL_RECOGNITION/keypoint.csv'
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number, *landmark_list])
    return

In [141]:
def draw_info(image, mode, number,label):
    mode_string = ['Logging Key Point']
    cv2.putText(image, label, (10, 130),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1,
                   cv2.LINE_AA)
    if 1 <= mode <= 2:
        cv2.putText(image, "MODE:" + mode_string[mode - 1], (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1,
                   cv2.LINE_AA)
        if 0 <= number <= 9:
            cv2.putText(image, "NUM:" + str(number), (10, 110),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1,
                       cv2.LINE_AA)
    return image

In [142]:
answer_labels= {0: 'Zero', 1: 'One', 2: 'Two' }

In [160]:

cap = cv2.VideoCapture(0)
mode = 0

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        #process key 
        key = cv2.waitKey(10)
        if key == 27:  # ESC
            break
        number, mode = select_mode(key, mode)
        print(number,mode)
        
        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        #get keypoints
        result_test = extract_keypoints(results)
        
        
        #collecting dataset to csv file
        logging_csv(number, mode, result_test)
        
        
        ans=np.argmax(np.squeeze(model.predict(np.reshape(result_test,(-1,1,84)))))
        
        
        #draw info to screen
        image = draw_info(image,mode,number,answer_labels[ans])
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

-1 0
1/1 [==============================] - 0s 17ms/step
-1 0
1/1 [==============================] - 0s 7ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 7ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 7ms/step
-1 0
1/1 [==============================] - 0s 8ms/step
-1 0
1/1 [==============================] - 0s 7ms/step
-1 0
1/1 [==============================] - 0s 7ms/step
-1 0
1/1 [==============================] - 0s 7ms/step
-1 0
1/1 [==============================] - 0s 7ms/step
-1 0
1/1 [==============================] - 0s 

In [143]:
df = pd.read_csv('/Users/richamodi/Desktop/ISL_RECOGNITION/keypoint.csv',header=None)
df.rename(columns={0: 'Lables'}, inplace=True)

In [144]:
df.head()

,Lables,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.310528,0.410676,0.229783,0.443003,0.264561,0.394034,0.294400,0.394338,0.314931,0.408772
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.516678,0.253716,0.443870,0.250322,0.491238,0.208929,0.514774,0.224221,0.526797,0.249577
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.616224,0.537571,0.525330,0.548476,0.580832,0.518305,0.604194,0.536024,0.616016,0.558417
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.622160,0.542727,0.526566,0.552323,0.581206,0.518977,0.605239,0.538634,0.617259,0.562745
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.220610,0.283440,0.145818,0.287857,0.171262,0.230922,0.200185,0.236365,0.218889,0.261125


In [145]:
X=df[df.columns[1:]].values
y=df[df.columns[0]].values

In [146]:
from sklearn.model_selection import train_test_split

In [147]:
X_train,X_temp,y_train, y_temp = train_test_split(X,y,random_state= 0 , test_size= 0.4)
X_val,X_test, y_val, y_test = train_test_split(X_temp,y_temp, test_size = 0.5, random_state = 0)

In [148]:
X.shape , X_train.shape,y.shape, y_train.shape, y_val.shape, X_test.shape, X_val.shape

((92, 84), (55, 84), (92,), (55,), (18,), (19, 84), (18, 84))

In [149]:
X_train_2 = np.reshape(X_train,(-1,1,84))
y_train_2 = np.reshape(y_train,(-1,1))

X_val_2 = np.reshape(X_val, (-1,1,84))
y_val_2 = np.reshape(y_val, (-1,1))

X_test_2 = np.reshape(X_test, (-1,1,84))
y_test_2 = np.reshape(y_test, (-1,1))

In [150]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [151]:
NUM_CLASS=3

In [153]:
model = Sequential()
model.add(InputLayer((1,84)))
model.add(LSTM(16,return_sequences= True))
model.add(LSTM(32))
model.add(Dense(16,'relu'))
model.add(Dropout(0.2))
model.add(Dense(32,'relu'))

model.add(Dense(16,'relu'))
model.add(Dense(NUM_CLASS,'softmax'))

In [154]:
#cp_callback = tf.keras.callbacks.ModelCheckpoint(
 #   'model1/', verbose=1, save_weights_only=False)

#es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [156]:
model.compile(optimizer = Adam(learning_rate = 0.01), metrics = ['accuracy'], loss = SparseCategoricalCrossentropy())
model.fit(X_train_2,y_train_2 , batch_size = 16, epochs = 20, validation_data = (X_val_2,y_val_2))

Epoch 1/20
4/4 [==============================] - 2s 104ms/step - loss: 0.2785 - accuracy: 0.9636 - val_loss: 1.0270e-04 - val_accuracy: 1.0000
Epoch 2/20
4/4 [==============================] - 0s 5ms/step - loss: 0.0498 - accuracy: 0.9818 - val_loss: 0.2578 - val_accuracy: 0.9444
Epoch 3/20
4/4 [==============================] - 0s 5ms/step - loss: 0.3122 - accuracy: 0.9455 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 4/20
4/4 [==============================] - 0s 5ms/step - loss: 0.0027 - accuracy: 1.0000 - val_loss: 0.0084 - val_accuracy: 1.0000
Epoch 5/20
4/4 [==============================] - 0s 5ms/step - loss: 0.0867 - accuracy: 0.9273 - val_loss: 5.7491e-04 - val_accuracy: 1.0000
Epoch 6/20
4/4 [==============================] - 0s 5ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 1.0465 - val_accuracy: 0.7778
Epoch 7/20
4/4 [==============================] - 0s 5ms/step - loss: 0.2157 - accuracy: 0.9455 - val_loss: 0.2694 - val_accuracy: 0.9444
Epoch 8/20
4/4 [========

In [157]:
model.evaluate(X_test_2,y_test_2)

1/1 [==============================] - 0s 17ms/step - loss: 0.0010 - accuracy: 1.0000


[0.001034931861795485, 1.0]